In [4]:
from __future__ import print_function,division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizer_v2.adam import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np



In [9]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        #build and compile discrimianatro
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        #build the gen
        self.generator = self.build_generator()

        #the gen takes noise as input ans gen imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        #for the combined model we will only train the gne
        self.discriminator.trainable = False

        #the discriminator takes genatred images as ninput asnd determines validity
        validity = self.discriminator(img)

        #the combined model(stacked gen and dscirimi)
        #trains the gen to fool the dscirmin
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer =optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        #load datset
        (X_train, _),(_,_) = mnist.load_data()

        #rescale -1 to 1
        X_train = X_train/127.5-1.
        X_train = np.expand_dims(X_train, axis=3)

        #advesarifal gts
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            #train discrimiantro

            #selec random bathco of imgaes
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0,1,(batch_size, self.latent_dim))

            #genea batch of new imgages

            gen_imgs = self.generator.predict(noise)

            #train the discrimianatro
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5* np.add(d_loss_real, d_loss_fake)

            #train_gen
            noise = np.random.normal(0,1, (batch_size, self.latent_dim))

            #train the gen (to have th ediscirmato label sampels as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            #plot the progress
            print('%d [D loss: %f, acc: %.2f%%] [G loss: %f' %(epoch, d_loss[0], 100*d_loss[1], g_loss))

            # if at save interval -> save gen image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5,5
        noise = np.random.normal(0,1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        #rescale images 0-1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)

        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt +=1
        fig.savefig('D:/OneDrive - 한국방송통신대학교/1_total_beat/study/test_184/kcow_gan/images/%d.png' %epoch)
        plt.close()

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

        

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_28 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_29 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr